**Goal:** Predict "log-error"

**Hypothesis:**

In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#wrangling
import pandas as pd
import numpy as np

#explore
import scipy.stats as stats

#visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


#default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

import env
import acquire
import prepare
# import summarize

### Acquire & Summarize


Acquired zillow data using acquire.py (sequel query in this file)

In [2]:
# df = acquire.get_zillow_data()
# df.head()

,tax_rate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,taxvaluedollarcnt,yearbuilt,landtaxvaluedollarcnt,taxdelinquencyflag,taxdelinquencyyear,logerror,transactions
0,0.01,3.50,4.00,"3,100.00","6,059.00",2.00,633.00,"33,634,931.00","-117,869,207.00","4,506.00",nan,nan,"1,023,282.00","1,998.00","537,569.00",None,nan,0.03,1
1,0.01,1.00,2.00,"1,465.00","6,111.00",1.00,0.00,"34,449,266.00","-119,281,531.00","12,647.00",nan,nan,"464,000.00","1,967.00","376,000.00",None,nan,0.06,1
2,0.01,2.00,3.00,"1,243.00","6,059.00",2.00,440.00,"33,886,168.00","-117,823,170.00","8,432.00",1.00,nan,"564,778.00","1,962.00","479,489.00",None,nan,0.01,1
3,0.01,3.00,4.00,"2,376.00","6,037.00",nan,nan,"34,245,180.00","-118,240,722.00","13,038.00",1.00,nan,"145,143.00","1,970.00","36,225.00",None,nan,-0.10,1
4,0.01,3.00,4.00,"2,962.00","6,037.00",nan,nan,"34,145,202.00","-118,179,824.00","63,000.00",1.00,nan,"773,303.00","1,950.00","496,619.00",None,nan,-0.00,1


In [ ]:
# df.transactiondate = pd.to_datetime(df.transactiondate, format='%Y-%m-%d')
# df = df.sort_values("transactiondate").drop_duplicates('parcelid',keep='last') 

2. Summary of zillow data (summary stats, info, dtypes, shape, distributions)

In [4]:
df.shape

(52169, 19)

In [5]:
df["county_name"] = df["fips"].map({6037: "Los Angeles", 6059: "Orange", 6111: "Ventura"})

In [6]:
df.to_csv(r'zillow_dataframe.csv')

In [8]:
df = pd.read_csv('zillow_dataframe.csv')

In [15]:
df.head()

,Unnamed: 0,tax_rate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,garagecarcnt,garagetotalsqft,latitude,longitude,...,poolcnt,poolsizesum,taxvaluedollarcnt,yearbuilt,landtaxvaluedollarcnt,taxdelinquencyflag,taxdelinquencyyear,logerror,transactions,county_name
0,0,0.01,3.50,4.00,"3,100.00","6,059.00",2.00,633.00,"33,634,931.00","-117,869,207.00",...,nan,nan,"1,023,282.00","1,998.00","537,569.00",NaN,nan,0.03,1,Orange
1,1,0.01,1.00,2.00,"1,465.00","6,111.00",1.00,0.00,"34,449,266.00","-119,281,531.00",...,nan,nan,"464,000.00","1,967.00","376,000.00",NaN,nan,0.06,1,Ventura
2,2,0.01,2.00,3.00,"1,243.00","6,059.00",2.00,440.00,"33,886,168.00","-117,823,170.00",...,1.00,nan,"564,778.00","1,962.00","479,489.00",NaN,nan,0.01,1,Orange
3,3,0.01,3.00,4.00,"2,376.00","6,037.00",nan,nan,"34,245,180.00","-118,240,722.00",...,1.00,nan,"145,143.00","1,970.00","36,225.00",NaN,nan,-0.10,1,Los Angeles
4,4,0.01,3.00,4.00,"2,962.00","6,037.00",nan,nan,"34,145,202.00","-118,179,824.00",...,1.00,nan,"773,303.00","1,950.00","496,619.00",NaN,nan,-0.00,1,Los Angeles


In [18]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
# df.info()

In [ ]:
# df.describe()

In [ ]:
# df.dtypes

In [ ]:
# pd.DataFrame(df.columns)

In [19]:
df = prepare.handle_missing_values(df)

In [20]:
df.isnull().sum()

tax_rate                          5
bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet      8
fips                              0
latitude                          0
longitude                         0
lotsizesquarefeet               354
taxvaluedollarcnt                 1
yearbuilt                        40
landtaxvaluedollarcnt             1
logerror                          0
transactions                      0
county_name                       0
dtype: int64

In [21]:
# cols_to_remove = ["unitcnt", "propertyzoningdesc", "heatingorsystemdesc", 
#                   "heatingorsystemtypeid", "buildingqualitytypeid", "assessmentyear",
#                   "calculatedbathnbr", "finishedsquarefeet12", "propertylandusedesc", 
#                   "propertylandusetypeid", "rawcensustractandblock" , "regionidcity",
#                   "state", "id", "fullbathcnt", "roomcnt", 'parcelid', 'taxvaluedollarcnt', 
#                   'structuretaxvaluedollarcnt', 'taxamount', 'transactiondate','transactions', 
#                   'calculatedfinishedsquarefeet', 'landtaxvaluedollarcnt', 'county', 'propertycountylandusecode']

In [22]:
# df['price_per_sq_ft'] = df.taxvaluedollarcnt/df.calculatedfinishedsquarefeet
# df['yard_sq_ft'] = df.lotsizesquarefeet - df.calculatedfinishedsquarefeet

In [ ]:
# df = prepare.remove_columns(df, cols_to_remove)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
cols = ["censustractandblock",  "fips", "regionidcounty", "regionidzip", "yearbuilt"]

In [ ]:
df = prepare.numeric_to_category(df, cols)

In [ ]:
df.dtypes

In [ ]:
df.latitude = df.latitude / 1_000_000 
df.longitude = df.longitude / 1_000_000 

In [ ]:
from sklearn.cluster import KMeans

X = df

kmeans = KMeans(n_clusters=5)
kmeans.fit(X)

X['cluster'] = pd.Series(kmeans.predict(X)).astype(str)

In [ ]:
X = X.dropna()

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(X, test_size=.30, random_state=123)

In [ ]:
train.shape

In [ ]:
train.describe().T

In [ ]:
X_train = train.drop(columns="logerror")

y_train = train[["logerror"]]

X_test = test.drop(columns="logerror")

y_test = test[["logerror"]]

In [ ]:
# df_nums_train = train.select_dtypes(exclude="category")

# df_nums_test = test.select_dtypes(exclude="category")

# df_nums_train.shape

In [ ]:
# x_df_nums_train = df_nums_train.drop(columns="logerror")

# y_df_nums_train = df_nums_train[["logerror"]]

# x_df_nums_test = df_nums_test.drop(columns="logerror")

# y_df_nums_test = df_nums_test[["logerror"]]

# x_df_nums_train.head().T

In [ ]:
# y_df_nums_train.head().T

## Encoding - encoding the 5 clusters made from the features

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train.head()

In [ ]:
def encode(X_train, X_test, col_name):
    
    encoded_values = sorted(list(X_train[col_name].unique()))

    # Integer Encoding
    int_encoder = LabelEncoder()
    X_train.encoded = int_encoder.fit_transform(X_train[col_name])
    X_test.encoded = int_encoder.transform(X_test[col_name])

    # create 2D np arrays of the encoded variable (in train and test)
    X_train_array = np.array(X_train.encoded).reshape(len(X_train.encoded),1)
    X_test_array = np.array(X_test.encoded).reshape(len(X_test.encoded),1)

    # One Hot Encoding
    ohe = OneHotEncoder(sparse=False, categories='auto')
    X_train_ohe = ohe.fit_transform(X_train_array)
    X_test_ohe = ohe.transform(X_test_array)

    # Turn the array of new values into a data frame with columns names being the values
    # and index matching that of train/test
    # then merge the new dataframe with the existing train/test dataframe
    X_train_encoded = pd.DataFrame(data=X_train_ohe,
                            columns=encoded_values, index=X_train.index)
    X_train = X_train.join(X_train_encoded)

    X_test_encoded = pd.DataFrame(data=X_test_ohe,
                               columns=encoded_values, index=X_test.index)
    X_test = X_test.join(X_test_encoded)

    return X_train, X_test, ohe

In [ ]:
train_cluster, test_cluster, ohe = encode(X_train, X_test, 'cluster')

In [ ]:
train_cluster.head()

### Model df - Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
train_cluster = train_cluster.drop(columns='cluster')

In [ ]:
test_cluster = test_cluster.drop(columns='cluster')

In [ ]:
rf = RandomForestRegressor(n_estimators=100)

In [ ]:
rf.fit(train_cluster, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = rf.predict(train_cluster)
print(f'root mean squared error = {mean_squared_error(y_train, y_pred)**1/2}')

In [ ]:
y_pred = rf.predict(test_cluster)
print(f'root mean squared error = {mean_squared_error(y_test, y_pred)**1/2}')

# Below is previous cleaning work.  ignore

- Number of rows with missing values

In [ ]:
number_missing = df.isnull().sum()

- percent of total rows that have missing values

In [ ]:
pct_missing = (df.isnull().sum())/df.shape[0]

In [ ]:
nulls_by_column_df = pd.DataFrame({'num_rows_missing': number_missing, 'pct_rows_missing': pct_missing})

In [ ]:
def nulls_by_col(df):
    number_missing = df.isnull().sum()
    pct_missing = (df.isnull().sum())/df.shape[0]
    rows_missing_df = pd.DataFrame({'num_rows_missing': number_missing, 'pct_rows_missing': pct_missing})
    return nulls_by_column_df

In [ ]:
nulls_by_column_df

In [ ]:
df.fips.unique()

In [ ]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [ ]:
num_cols_missing = df.isnull().sum(axis=1)
pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()

In [ ]:
rows_missing

### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

#### Single Unit Properties (as defined by Jeff Hutchins)

Single Family Residential = 52320

Residential General = 37

Rural Residence = 0

Mobile Home = 74

Manufactured, Modular, Prefabricated Homes = 58

Inferred Single Family Residential = 0

Bungalow = 0

## Going to create a new column called price_per_sq_ft, use a clustering method called K-means clustering to find clusters of prices, and compare it to latitude and longitude points.

In [ ]:
df.head()

In [ ]:
df_subset = df[['calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'latitude', 'longitude']]
df_subset.head()

In [ ]:
df_subset['price_per_sq_ft'] = df_subset.taxvaluedollarcnt/df_subset.calculatedfinishedsquarefeet

In [ ]:
df_subset.head()

In [ ]:
df_subset = df_subset.dropna()

In [ ]:
df_subset.isnull().sum()

This graph shows the shape of the combination of all three counties.  The different colors represent the clusters.  The clusters are based on price per square foot, latitude, and longitude.

Used KMeans clustering on price per square foot, latitude, and longitude.  Then used the clusters as a hue to map it onto a 2D graph with longitude on the x-axis and latitude on the y-axis.  

Notes:

**Try to insert a slide for n_clusters**

**Try adding more variables**

In [ ]:
from sklearn.cluster import KMeans

X = df_subset[['price_per_sq_ft', 'latitude', 'longitude']]

kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

X['cluster'] = 'clusters ' + pd.Series(kmeans.predict(X)).astype(str)

sns.relplot(data=X, hue='cluster', x='longitude', y='latitude')

In [ ]:
print(kmeans.labels_)
print(kmeans.inertia_)
print(kmeans.n_iter_)
print(kmeans.cluster_centers_)

In [ ]:
df_subset.head()

In [ ]:
from sklearn.cluster import KMeans

X = df_subset[['price_per_sq_ft']]

kmeans = KMeans(n_clusters=8)
kmeans.fit(X)

In [ ]:
print(kmeans.labels_)

print(kmeans.cluster_centers_)

In [ ]:
X['cluster'] = 'clusters ' + pd.Series(kmeans.predict(X)).astype(str)

In [ ]:
X['latitude'] = df_subset['latitude']

In [ ]:
X['longitude'] = df_subset['longitude']

This graph shows the shape of the combination of all three counties.  The different colors represent the clusters.  The clusters are based on price per square foot.

Used KMeans clustering on price per square foot.  Then used the clusters as a hue to map it onto a 2D graph with longitude on the x-axis and latitude on the y-axis.  

In [ ]:
sns.relplot(data=X, hue='cluster', x='longitude', y='latitude')

In [ ]:
df[['latitude', 'longitude']].head()

In [ ]:
# df['latitude'].value_counts()

In [ ]:
# df['longitude'].value_counts()
# pd.DataFrame(pd.cut(df['longitude'], bins=[-120_000_000, -119_000_000, -118_000_000, -117_000_000, -116_000_000]))

Also try lot size minus sq ft of house

In [ ]:
df.head()

**Creating a new feature using the lot size square footage minus the square footage of the house**

In [ ]:
df['yard_square_footage'] = (df['lotsizesquarefeet'] - df['calculatedfinishedsquarefeet'])

In [ ]:
df.head()

**Project Planning** graph ideas, hypotheses, doodles, data dictionary

**Acquire**

**Prep** - Nulls, outliers, visualie distribution, drop variables

**Split Data**

**Impute** - Don't want to use test data as evidence of what to impute.  Use train to find imputer then transform train and test

**Scale** - can choose to scale variables differently. Just choose between 0 and 1.

**explore, visualize, clustering, stats, testing, etc.** (in no particular order)

Audience is the class.  Work with partner.  Choose one of three ways to apply clustering.  Zillow Data.  Go from end to end.  Share the highlights of the discoveries, what we uncovered, exploration, modeling.  What we've learned and how it relates to data science.

What we learned as it relates to domain, what we learned as it relates to data science, and what we learned as it relates to clustering.

Trying to predict log error because we want to help zillow improve their zestimate.  Which features drive the error. Trying to predict a continuous variable.  Predict the target.

If we encode, do so after clustering.  Have to encode your clusters.  Also makes it easier to visualize the data.

1. Audience: class, fellow learners
2. Deliverable: Notebook with supporting files
    - clean, easy to read
    - separate modules (acq, prep, etc.)
3. Team of 2
4. Clustering (are these clusters drivers of the target?)
    - Clusters for features
    - Clusters for explorations
    - Clustering target variable (binning)
5. Analysis/takeaways with a model.  What is the best model you made?  

Has to have:
 - statistical testing
 - visualizations of clusters
 - clusters
 - models
 - summary


**Visualizations**

continuous vs continuous, relplot

discrete vs continuous, t-test (group pool or not pool and compare prices)

two discrete, chi-squared test, pandas crosstabs, clustering

Do stuff, Learn Clustering, Model